In [69]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('../data/reviews.csv', encoding='utf-8')
print('number of reviews:',len(reviews))
reviews.info()
reviews.head()

number of reviews: 18393
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 2 columns):
reviewid    18393 non-null int64
content     18383 non-null object
dtypes: int64(1), object(1)
memory usage: 287.5+ KB


,reviewid,content
0,22703,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,"Eight years, five albums, and two EPs in, the ..."
2,22659,Minneapolis’ Uranium Club seem to revel in bei...
3,22661,Kleenex began with a crash. It transpired one ...
4,22725,It is impossible to consider a given release b...


In [70]:
meta = pd.read_csv('../data/meta.csv')
meta.info()
meta.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 13 columns):
reviewid          18393 non-null int64
title             18391 non-null object
artist            18391 non-null object
url               18393 non-null object
score             18393 non-null float64
best_new_music    18393 non-null int64
author            18393 non-null object
author_type       14487 non-null object
pub_date          18393 non-null object
pub_weekday       18393 non-null int64
pub_day           18393 non-null int64
pub_month         18393 non-null int64
pub_year          18393 non-null int64
dtypes: float64(1), int64(6), object(6)
memory usage: 1.8+ MB


,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017


In [71]:
meta.drop(['title','artist', 'url', 'author', 'author_type', 'best_new_music', 'pub_date', 'pub_weekday', 'pub_day', 'pub_month', 'pub_year'], axis=1, inplace=True)
meta.info()
meta.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 2 columns):
reviewid    18393 non-null int64
score       18393 non-null float64
dtypes: float64(1), int64(1)
memory usage: 287.5 KB


,reviewid,score
0,22703,9.3
1,22721,7.9
2,22659,7.3
3,22661,9.0
4,22725,8.1


In [87]:
df = pd.merge(reviews, meta, on='reviewid')
df.drop('reviewid', axis=1, inplace=True)
df.dropna(inplace=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18391 entries, 0 to 18400
Data columns (total 2 columns):
content    18391 non-null object
score      18391 non-null float64
dtypes: float64(1), object(1)
memory usage: 431.0+ KB


,content,score
0,"“Trip-hop” eventually became a ’90s punchline,...",9.3
1,"Eight years, five albums, and two EPs in, the ...",7.9
2,Minneapolis’ Uranium Club seem to revel in bei...,7.3
3,Kleenex began with a crash. It transpired one ...,9.0
4,It is impossible to consider a given release b...,8.1


In [109]:
NUM_CLASSES = 3
labels = np.arange(NUM_CLASSES)
df['target'] = pd.cut(df.score, 3, labels=labels)
df.head()

,content,score,target
0,"“Trip-hop” eventually became a ’90s punchline,...",9.3,2
1,"Eight years, five albums, and two EPs in, the ...",7.9,2
2,Minneapolis’ Uranium Club seem to revel in bei...,7.3,2
3,Kleenex began with a crash. It transpired one ...,9.0,2
4,It is impossible to consider a given release b...,8.1,2


In [129]:
#finally lets reduce the number of articles we're working with
df = df.sample(n=5000, random_state=42)

In [130]:
# this is an array of the reviews
texts = df.content.values

In [131]:
texts[0]

"Big Star never completed their third album. In fact, it’s likely\xa0that the music collected on Omnivore’s triple-disc box set Complete Third was never intended to be a Big Star album. Alex Chilton maintained as late as 2007 that he and drummer Jody Stephens never considered these 1974 sessions a Big Star project, a testament supported by the fact that none of the tapes in Ardent Studios were labeled with name “Big Star.” Instead, they were credited to Alex Chilton alone, Alex & Jody, and Sister Lovers, a punning reference to the fact the pair were dating sisters at the time. \xa0Sister Lovers also provided Rykodisc with a subtitle for these recordings when they issued Third/Sister Lovers on CD in 1992, marking the first time a label attempted to seriously piece the puzzle of Big Star Third together. Prior to that, the album came out under a variety of titles—3rd, The Third Album, Big Star’s 3rd: Sister Lovers, Sister Lovers (The Third Album), with the alleged provisional title Beale 

In [132]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [135]:
%%time
NUM_WORDS = 5000
MAX_ART_LEN = 1000

tkn = Tokenizer(num_words=NUM_WORDS)
tkn.fit_on_texts(texts)

sequences = tkn.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=MAX_ART_LEN)

y = np.asanyarray(df.target)
y_ohe = keras.utils.to_categorical(y)
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', y_ohe.shape)
print(np.max(X))

Shape of data tensor: (5000, 1000)
Shape of label tensor: (5000, 3)
4999
Wall time: 6.71 s


## RNN stuff


In [136]:
from sklearn.model_selection import train_test_split
#from keras.utils.

X_train, X_test, y_train, y_test = train_test_split(X, y_ohe, test_size=0.2)

In [137]:
from keras.models import Sequential, Input, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers.embeddings import Embedding

EMBED_SIZE = 50
input_holder = Input(shape=(X_train.shape[1], ))
shared_embed = Embedding(NUM_WORDS, 
                EMBED_SIZE, 
                input_length=MAX_ART_LEN)(input_holder)

x = LSTM(50,dropout=0.2, recurrent_dropout=0.2)(shared_embed)
x = Dense(NUM_CLASSES, activation='sigmoid')(x)
rnn=Model(inputs=input_holder,outputs=x)
rnn.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1000, 50)          250000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 153       
Total params: 270,353
Trainable params: 270,353
Non-trainable params: 0
_________________________________________________________________
None


In [138]:
rnn.fit(X_train, y_train, batch_size=128, epochs=3, validation_data=[X_test, y_test])

Train on 4000 samples, validate on 1000 samples
Epoch 1/3
4000/4000 [==============================] - 97s - loss: 0.7728 - acc: 0.6912 - val_loss: 0.6737 - val_acc: 0.7020
Epoch 2/3
4000/4000 [==============================] - 92s - loss: 0.6938 - acc: 0.7013 - val_loss: 0.6784 - val_acc: 0.7020
Epoch 3/3
4000/4000 [==============================] - 90s - loss: 0.6934 - acc: 0.7013 - val_loss: 0.6721 - val_acc: 0.7020
